In [264]:
import numpy as np
from numpy import tanh

In [336]:
# activation function 1
def sigmoid(z):
    return 1.0/(1+np.exp(-z))
# activation function 2
def softmax(z):
    return np.exp(z)/np.sum(np.exp(z),axis=1,keepdims=True)
    

class MultilayerPerceptron:
    
    def __init__(self,input_layer,hidden_layers,output_layer):
        np.random.seed(0)
        model={ 'weights': {}, 'bias': {} }
        layers = [input_layer]+hidden_layers+[output_layer]
        for i in range(len(layers)-1):
            W=np.random.randn(layers[i],layers[i+1])
            b=np.random.randn(1,layers[i+1])
            model['weights'][i]=W
            model['bias'][i]=b
        
        self.model=model
        self.no_of_layers=len(layers)
    
    def forwardPropagation(self,X):
        activations={}
        activations[0]=X
        n=self.no_of_layers
        for i in range(1,n):
            z = np.dot(activations[i-1],self.model['weights'][i-1]) + self.model['bias'][i-1]
            if i<n-1:
                activations[i]=tanh(z)
            else:
                activations[n-1]=softmax(z)   
        
        return activations

In [337]:
from sklearn.datasets import make_classification
X,Y=make_classification(n_samples=100,n_features=4,n_informative=4,n_redundant=0,random_state=9)
mlp=MultilayerPerceptron(4,[5,5],2)
activations=mlp.forwardPropagation(X)

In [338]:
print(activations[3])

[[0.9156633  0.0843367 ]
 [0.79955636 0.20044364]
 [0.81856386 0.18143614]
 [0.92204287 0.07795713]
 [0.97691905 0.02308095]
 [0.83048693 0.16951307]
 [0.98368514 0.01631486]
 [0.98360537 0.01639463]
 [0.82359206 0.17640794]
 [0.98079034 0.01920966]
 [0.53332714 0.46667286]
 [0.88026661 0.11973339]
 [0.95457111 0.04542889]
 [0.95932282 0.04067718]
 [0.78489118 0.21510882]
 [0.76249868 0.23750132]
 [0.8851822  0.1148178 ]
 [0.87222206 0.12777794]
 [0.97748972 0.02251028]
 [0.36349437 0.63650563]
 [0.81110409 0.18889591]
 [0.561519   0.438481  ]
 [0.39861643 0.60138357]
 [0.50370041 0.49629959]
 [0.8098757  0.1901243 ]
 [0.78997256 0.21002744]
 [0.3344725  0.6655275 ]
 [0.48340304 0.51659696]
 [0.95203069 0.04796931]
 [0.67460284 0.32539716]
 [0.49528384 0.50471616]
 [0.8924458  0.1075542 ]
 [0.96390192 0.03609808]
 [0.97317889 0.02682111]
 [0.98309757 0.01690243]
 [0.50816246 0.49183754]
 [0.50417992 0.49582008]
 [0.39714525 0.60285475]
 [0.97405156 0.02594844]
 [0.96754917 0.03245083]
